**LSTM-based NMT  SAMPLES**

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

data = {
    'english': ['hello', 'how are you', 'good morning', 'thank you'],
    'hindi': ['नमस्ते', 'आप कैसे हैं', 'शुभ प्रभात', 'धन्यवाद']
}
df = pd.DataFrame(data)

tokenizer_en = Tokenizer()
tokenizer_hi = Tokenizer()

tokenizer_en.fit_on_texts(df['english'])
tokenizer_hi.fit_on_texts(df['hindi'])

sequences_en = tokenizer_en.texts_to_sequences(df['english'])
sequences_hi = tokenizer_hi.texts_to_sequences(df['hindi'])


max_length_en = max(len(seq) for seq in sequences_en)
max_length_hi = max(len(seq) for seq in sequences_hi)

X = pad_sequences(sequences_en, maxlen=max_length_en, padding='post')
y = pad_sequences(sequences_hi, maxlen=max_length_hi, padding='post')

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer_en.word_index)+1, output_dim=64, input_length=max_length_en))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tokenizer_hi.word_index)+1, activation='softmax')))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


y = np.expand_dims(y, -1)
model.fit(X, y, epochs=100)


model.save('lstm_nmt_model.h5')

import pickle
with open('tokenizer_en.pkl', 'wb') as file:
    pickle.dump(tokenizer_en, file)

with open('tokenizer_hi.pkl', 'wb') as file:
    pickle.dump(tokenizer_hi, file)
